In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pathlib
import urllib.request

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.transforms
import scipy.ndimage.measurements
import scipy.interpolate
import scipy.optimize

import imageio

import pymedphys._mocks.profiles
import pymedphys._wlutz.findfield
import pymedphys._wlutz.createaxis
import pymedphys._wlutz.interppoints

In [ ]:
x_centre=-5.597107643053333
y_centre=14.88603370568775
x_edge=10
y_edge=10
penumbra=0.5
actual_rotation=263.3971055572235



actual_rotation = actual_rotation
edge_lengths = [x_edge, y_edge]
# penumbra = 0.5
actual_centre = [x_centre, y_centre]

field = pymedphys._mocks.profiles.create_rectangular_field_function(actual_centre, edge_lengths, penumbra, actual_rotation)

In [ ]:
(180 - 90) % 90

In [ ]:
x = np.arange(-50, 50, 0.1)
y = np.arange(-50, 50, 0.1)
xx, yy = np.meshgrid(x, y)
zz = field(xx, yy)
plt.pcolormesh(xx, yy, zz)
plt.axis('equal')

In [ ]:
initial_centre = pymedphys._wlutz.findfield._initial_centre(x, y, zz)
centre, rotation = pymedphys._wlutz.findfield.field_centre_and_rotation_refining(field, edge_lengths, penumbra, initial_centre)
centre, rotation

In [ ]:
while True:
    centre, rotation = pymedphys._wlutz.findfield.field_centre_and_rotation_refining(field, edge_lengths, penumbra, initial_centre)

In [ ]:
# rotation = 25.01728363860255

In [ ]:
def draw_by_diff(dx, dy, transform):
    draw_x = np.cumsum(dx)
    draw_y = np.cumsum(dy)

    draw_x, draw_y = pymedphys._wlutz.interppoints.apply_transform(draw_x, draw_y, transform)
    
    return draw_x, draw_y

In [ ]:
transform = matplotlib.transforms.Affine2D()
transform.rotate_deg(-rotation)
transform.translate(*centre)

In [ ]:
rotation_x_points = np.linspace(-edge_lengths[0]/2, edge_lengths[0]/2, 51)
rotation_y_points = np.linspace(-edge_lengths[1]/2, edge_lengths[1]/2, 61)

rot_xx_points, rot_yy_points = np.meshgrid(rotation_x_points, rotation_y_points)

rot_xx_points, rot_yy_points = pymedphys._wlutz.interppoints.apply_transform(rot_xx_points, rot_yy_points, transform)

In [ ]:
rotation_points = pymedphys._wlutz.interppoints.define_rotation_field_points(centre, edge_lengths, penumbra, rotation)

In [ ]:
rect_dx = [-edge_lengths[0]/2, 0, edge_lengths[0], 0, -edge_lengths[0]]
rect_dy = [-edge_lengths[1]/2, edge_lengths[1], 0, -edge_lengths[1], 0]

rect_crosshair_dx = [-edge_lengths[0]/2, edge_lengths[0], -edge_lengths[0], edge_lengths[0]]
rect_crosshair_dy = [-edge_lengths[1]/2, edge_lengths[1], 0, -edge_lengths[1]]

plt.figure(figsize=(10,10))
plt.pcolormesh(xx, yy, zz)
plt.plot(*draw_by_diff(rect_dx, rect_dy, transform), 'k', lw=2)
plt.plot(*draw_by_diff(rect_crosshair_dx, rect_crosshair_dy, transform), 'k', lw=0.5)

# plt.plot(rot_xx_points, rot_yy_points, '.')
plt.scatter(rotation_points[0], rotation_points[1], s=1)

plt.scatter(centre[0], centre[1], c='r', s=1)
plt.axis('equal')

In [ ]:
np.allclose(actual_centre, centre)

In [ ]:
np.allclose(actual_rotation, rotation)

In [ ]:
actual_rotation, rotation